In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Reading the dataset**

In [ ]:
df=pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

**Checking for null values**

In [ ]:
df.isnull().sum()

**Storing the features in X**

In [ ]:
X=df.drop(['id','diagnosis','Unnamed: 32'],axis=1)
X

**Storing the label in y**

In [ ]:
y=df['diagnosis']
y

**Encoding the label for better classification**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

# Encode label category
# Malignent-> 1
# Benign -> 0

dia_status=LabelEncoder()
dia_status

Transforming y into encoded label

In [ ]:
y=dia_status.fit_transform(y)
y

**Lets look at the correlations and do some feature selection**

In [ ]:
X.corr()

Seaborn heatmap for visualizing correlations

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure(figsize=(14,12))
sns.heatmap(X.corr(),annot=True,cmap=plt.cm.CMRmap_r,alpha=.9)
plt.tight_layout()

**Creating a function for finding the columns having correaltion beyond threshold**

In [ ]:
# eg. columns col1 and col2 have correaltion greater than threshold, 
# then we can remove one of the col1 or col2 to balance biasness
# the function return the columns to be removed

def correlation(dataset,threshold):
    col_corr=set()
    corr_matrix=dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])>threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
            

In [ ]:
# columns having correlation greater than 0.90
corr_features=correlation(X,.90)
corr_features

In [ ]:
# dropping columns
X=X[X.drop(corr_features,axis=1).columns]

In [ ]:
X

Data Standardisation:-

The idea behind StandardScaler is that it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.
In case of multivariate data, this is done feature-wise (in other words independently for each column of the data).
Given the distribution of the data, each value in the dataset will have the mean value subtracted, and then divided by the standard deviation of the whole dataset (or feature in the multivariate case)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X)


In [ ]:
X=scaler.transform(X)

In [ ]:
X.shape

**Splitting the data for training and testing in 4:1 ratio**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=101)

**SVM with default hyperparameter**

In [ ]:
from sklearn.svm import SVC

clf=SVC()
clf.fit(X_train,y_train)
print('Accuracy : ', clf.score(X_test,y_test))

**Default Linear kernel**

In [ ]:
clf=SVC(kernel='linear')
clf.fit(X_train,y_train)
print('Accuracy : ', clf.score(X_test,y_test))

**Default RBF kernel**

In [ ]:
clf=SVC(kernel='rbf')
clf.fit(X_train,y_train)
print('Accuracy : ', clf.score(X_test,y_test))

**Default Polynomial kernel**

In [ ]:
clf=SVC(kernel='poly')
clf.fit(X_train,y_train)
print('Accuracy : ', clf.score(X_test,y_test))

**Lets use GridSearchCV to find best parameter**

In [ ]:
svm=SVC()

In [ ]:
# we are providing the parameters in form of list of dictionaries

tuned_parameter=[{'C':[0.1,0.5,1,5,10,50,100],'kernel':['linear']},
                 {'gamma':[0.0001,0.001,0.01,0.1,1,10],'kernel':['rbf']},
                 {'degree':[1,2,3,4],'kernel':['poly']}]

In [ ]:
from sklearn.model_selection import GridSearchCV

model=GridSearchCV(estimator=svm,
                   param_grid=tuned_parameter,
                   cv=10,
                   n_jobs=-1)


In [ ]:
# fitting the training data

model.fit(X_train,y_train)

In [ ]:
# best score

model.best_score_

In [ ]:
# best parameters from the parameters that we provided

model.best_params_

In [ ]:
# testing our model
model.score(X_test,y_test)